In [ ]:
#codigo_canhero = 6457
fecha_inicio = '2021-10-25'
fecha_final = '2021-10-28'
cod_ca = 772

In [ ]:
from werkzeug.wrappers import Request, Response
from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime

import collections
from amigocloud import AmigoCloud

import collections

In [ ]:
amigocloud = AmigoCloud(token='')
amigocloud

In [ ]:
query_reporte = {'query': 'select * from dataset_272975 where (fecha_registro between \'{fecha_ini}\' and \'{fecha_fin}\') and cast(SPLIT_PART(canhero, \' / \', 1) as integer) = {cod}'.format(fecha_ini=fecha_inicio, fecha_fin=fecha_final, cod=cod_ca)}
reporte_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/30837/sql', query_reporte)
reporte_data = reporte_select['data']

#credito_data['insp_fech_inf'] = datetime.now().strftime('%d/%m/%Y')
#date_update = credito_data['date_update']
#credito_data['date_update'] = datetime.strptime(date_update[:10], '%Y-%m-%d').strftime('%d/%m/%Y')
#credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())

reporte_data


#print(query_reporte)
#print('end')

In [ ]:
for i in reporte_data:
    query = {'query': 'select detalle, estado, s3_filename, fo.amigo_id from dataset_276804 fo inner join gallery_37796 ga on ga.source_amigo_id = fo.amigo_id where fotografias=\'{id}\''.format(id=i['amigo_id'])}
    select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/30837/sql', query)
    data = select['data']
    i['obs']=data

In [ ]:
reporte_data

In [ ]:
import requests
import docxtpl
from docxtpl import DocxTemplate
from docx.shared import Mm

In [ ]:
for insp in reporte_data:
    doc = DocxTemplate("template/template01.docx")
    lista_fotos_inline = []
    for obs in insp['obs']:
        print(obs['amigo_id'] + '  ---  ' + obs['s3_filename'])
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + obs['s3_filename'])
        file = open('fotos/' + obs['amigo_id']+'.jpg', "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + obs['amigo_id']+'.jpg', width=Mm(70)), 'obs': obs['detalle'], 'estado':obs['estado']})
    data = collections.namedtuple("insp", insp.keys())(*insp.values())
    context = {'insp':data, 'obs':lista_fotos_inline}
    doc.render(context)
    doc.save('_' + str(insp['canhero'].replace('/','-')) + '-' + str(insp['propiedad'].replace('/','-')) + '.docx')